Rahmanda Afebrio Yuris Soesatyo - Chapter 16:Natural Language Processing with RNNs and Attention

1. Character RNN and Shakespeare Text Generation

Bab ini diawali dengan pembahasan Character-level RNN (Char-RNN), yaitu model rekuren yang dilatih untuk memprediksi karakter berikutnya dalam teks karya Shakespeare. Dengan mempelajari distribusi karakter, model dapat menghasilkan teks baru secara autoregresif, satu karakter demi satu, menyerupai gaya penulisan aslinya.

Karena ukuran korpus teks sangat besar—mencapai lebih dari satu juta karakter—data tidak diproses secara utuh. Sebagai gantinya, teks dipecah menjadi banyak window pendek menggunakan tf.data.Dataset.window() dan pendekatan truncated Backpropagation Through Time (BPTT). Contohnya, urutan sepanjang 100 karakter digunakan sebagai input, dengan karakter ke-101 sebagai target prediksi.

Penyusunan Dataset

Setelah window dibentuk, pipeline dataset umumnya melibatkan langkah-langkah berikut:

Flattening window menggunakan flat_map,

pengacakan data (shuffle),

pengelompokan ke dalam batch (batch),

pemisahan antara:

input (semua karakter kecuali yang terakhir),

target (semua karakter kecuali yang pertama).

Setiap karakter direpresentasikan dalam bentuk one-hot encoding menggunakan tf.one_hot.

Arsitektur Model

Model yang digunakan biasanya berupa:

GRU dua lapis,

diikuti oleh TimeDistributed(Dense) dengan fungsi aktivasi softmax.

Model dilatih menggunakan fungsi loss sparse_categorical_crossentropy untuk memprediksi distribusi probabilitas seluruh karakter (misalnya 39 karakter unik) pada setiap time step.

Stateless vs Stateful RNN

Pada stateless RNN, hidden state selalu di-reset ke nol pada setiap batch. Akibatnya, model hanya mampu menangkap ketergantungan temporal sepanjang window yang digunakan dalam BPTT.

Untuk mempelajari dependensi yang lebih panjang tanpa melakukan backpropagation pada seluruh teks, digunakan pendekatan stateful RNN. Pada metode ini, hidden state terakhir dari satu batch digunakan sebagai initial state untuk batch berikutnya.

Agar stateful RNN bekerja dengan benar, beberapa syarat harus dipenuhi:

batch data harus benar-benar berurutan,

window dibuat tanpa overlap (shift = n_steps),

batch size biasanya kecil (bahkan 1 untuk konfigurasi paling sederhana),

layer RNN didefinisikan dengan stateful=True dan
batch_input_shape = [batch_size, None, vocab_size],

hidden state di-reset secara eksplisit di awal setiap epoch menggunakan callback khusus.

In [ ]:

import tensorflow as tf
from tensorflow import keras
import numpy as np

# encoded: array 1D berisi ID karakter (0..max_id-1)
dataset_size = len(encoded)
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

n_steps = 100
window_length = n_steps + 1

dataset = dataset.window(window_length, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))

batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

max_id = 39  # jumlah karakter unik
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch)
)
dataset = dataset.prefetch(1)

model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True,
                     input_shape=[None, max_id],
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.GRU(128, return_sequences=True,
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.TimeDistributed(
        keras.layers.Dense(max_id, activation="softmax")
    )
])

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, epochs=20)

2. Stateful RNN for Longer Dependencies

Pada stateless RNN, hidden state selalu di-reset ke nol pada setiap batch. Konsekuensinya, model hanya mampu mempelajari ketergantungan yang terbatas pada panjang window yang di-unroll selama Backpropagation Through Time (BPTT).

Untuk menangkap ketergantungan jangka lebih panjang tanpa harus melakukan backpropagation pada seluruh urutan teks, diperkenalkan konsep stateful RNN. Pada pendekatan ini, hidden state akhir dari suatu batch dipertahankan dan digunakan kembali sebagai state awal untuk batch berikutnya, sehingga informasi dapat mengalir lintas batch.

Agar stateful RNN bekerja secara benar dan konsisten, beberapa syarat penting harus dipenuhi:

Batch harus berisi urutan yang benar-benar saling berlanjut, bukan hasil shuffle acak.
Dataset dibentuk menggunakan window non-overlap, dengan shift = n_steps.
Batch size kecil sangat dianjurkan (bahkan 1 untuk konfigurasi paling sederhana).
Layer RNN dikonfigurasi dengan stateful=True dan
batch_input_shape = [batch_size, None, vocab_size].
Hidden state harus di-reset secara eksplisit pada awal setiap epoch, biasanya menggunakan callback khusus.
Dengan pengaturan ini, stateful RNN mampu memodelkan struktur dan dependensi jangka panjang secara lebih efektif dibanding stateless RNN, tanpa biaya komputasi backpropagation yang berlebihan.

In [ ]:

batch_size = 1  # sederhana: satu sequence per batch
n_steps = 100
window_length = n_steps + 1

dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))
dataset = dataset.batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch)
)
dataset = dataset.prefetch(1)

model = keras.models.Sequential([
    keras.layers.GRU(
        128, return_sequences=True, stateful=True,
        dropout=0.2, recurrent_dropout=0.2,
        batch_input_shape=[batch_size, None, max_id]
    ),
    keras.layers.GRU(
        128, return_sequences=True, stateful=True,
        dropout=0.2, recurrent_dropout=0.2
    ),
    keras.layers.TimeDistributed(
        keras.layers.Dense(max_id, activation="softmax")
    )
])

class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs=None):
        self.model.reset_states()

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, epochs=50, callbacks=[ResetStatesCallback()])


3. Sentiment Analysis with Word-Level RNNs

Setelah membahas pemodelan pada tingkat karakter, bab ini beralih ke word-level RNN untuk tugas sentiment analysis, menggunakan dataset IMDb yang berisi sekitar 50.000 ulasan film.

Pada pendekatan ini, setiap kata direpresentasikan sebagai ID numerik, baik dengan memanfaatkan dataset IMDb bawaan Keras maupun dengan membangun vocabulary kustom menggunakan Tokenizer atau tf.lookup.StaticVocabularyTable. ID kata tersebut kemudian dipetakan ke vektor kontinu berdimensi rendah melalui Embedding layer, sehingga model dapat mempelajari representasi semantik kata secara end-to-end.

Untuk membangun vocabulary kustom, seluruh teks diproses satu kali untuk menghitung frekuensi kata (misalnya dengan Counter). Selanjutnya dipilih sejumlah kata paling sering muncul (misalnya 10.000 kata), sementara kata lain dipetakan ke bucket out-of-vocabulary (OOV).

Pipeline pemrosesan data secara keseluruhan adalah sebagai berikut:

dataset di-batch,
token teks di-preprocess,
token di-lookup menjadi ID,
urutan ID dimasukkan ke model RNN dua lapis berbasis GRU,
satu neuron output dengan aktivasi sigmoid digunakan untuk memprediksi sentimen positif atau negatif.
Pendekatan word-level ini memungkinkan model menangkap konteks semantik yang lebih kaya dibanding character-level modeling, sehingga umumnya memberikan performa yang lebih baik pada tugas analisis sentimen.

In [ ]:
import tensorflow_datasets as tfds
from collections import Counter
from tensorflow import keras
import tensorflow as tf

datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)

def preprocess(X_batch, y_batch):
    # lowercase & split jadi tokens bytes
    X_batch = tf.strings.regex_replace(X_batch, rb"", b" ")
    X_batch = tf.strings.regex_replace(X_batch, rb"[^a-zA-Z']", b" ")
    X_batch = tf.strings.lower(X_batch)
    return tf.strings.split(X_batch), y_batch

vocabulary = Counter()
for X_batch, y_batch in datasets["train"].batch(32).map(preprocess):
    for review in X_batch:
        vocabulary.update(list(review.numpy()))

vocab_size = 10000
truncated_vocabulary = [
    word for word, count in vocabulary.most_common()[:vocab_size]
]

words = tf.constant(truncated_vocabulary)
word_ids = tf.range(len(truncated_vocabulary), dtype=tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)
num_oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init, num_oov_buckets)

def encode_words(X_batch, y_batch):
    return table.lookup(X_batch), y_batch

train_set = datasets["train"].batch(32).map(preprocess)
train_set = train_set.map(encode_words).prefetch(1)

embed_size = 128
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size,
                           input_shape=[None]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.GRU(128),
    keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="adam",
              metrics=["accuracy"])
history = model.fit(train_set, epochs=5)

4. Masking, Pretrained Embeddings, and Variable-Length Sequences

Ulasan teks memiliki panjang yang bervariasi, sehingga perlu dilakukan padding agar dapat diproses dalam batch. Untuk mencegah model belajar dari token padding (biasanya direpresentasikan dengan ID 0), digunakan mekanisme masking, sehingga token tersebut diabaikan saat perhitungan hidden state dan loss.

Pendekatan paling sederhana adalah dengan mengaktifkan opsi
mask_zero=True pada Embedding layer. Dengan cara ini, Keras secara otomatis membuat mask untuk token bernilai nol dan meneruskannya ke layer berikutnya, seperti RNN atau TimeDistributed, yang mendukung masking.

Pada arsitektur yang lebih kompleks—misalnya kombinasi Conv1D dan GRU—masking sering perlu ditangani secara manual. Mask dapat dihitung menggunakan ekspresi seperti K.not_equal(inputs, 0) di dalam Lambda layer, kemudian diteruskan secara eksplisit ke layer GRU melalui Functional API.

Selain pendekatan berbasis RNN, bab ini juga memperkenalkan penggunaan pretrained sentence embeddings dari TensorFlow Hub sebagai alternatif yang lebih sederhana. Contohnya, model nnlm-en-dim50 dapat digunakan melalui hub.KerasLayer untuk:

menerima input berupa string mentah,
langsung menghasilkan embedding kalimat berdimensi tetap,
kemudian hanya memerlukan satu atau beberapa Dense layer untuk tugas sentiment analysis.
Pendekatan berbasis pretrained embeddings ini sering memberikan hasil yang kompetitif, terutama ketika data berlabel terbatas, serta secara signifikan menyederhanakan arsitektur model.

In [ ]:

from tensorflow import keras
import tensorflow.keras.backend as K
import tensorflow_hub as hub

# Masking manual di Functional API
vocab_size = 10000; num_oov_buckets = 1000; embed_size = 128
inputs = keras.layers.Input(shape=[None])
mask = keras.layers.Lambda(lambda x: K.not_equal(x, 0))(inputs)
z = keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size)(inputs)
z = keras.layers.GRU(128, return_sequences=True)(z, mask=mask)
z = keras.layers.GRU(128)(z, mask=mask)
outputs = keras.layers.Dense(1, activation="sigmoid")(z)
model_mask = keras.Model(inputs=inputs, outputs=outputs)

# Model dengan sentence embedding dari TF Hub
hub_layer = hub.KerasLayer(
    "https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1",
    dtype=tf.string, input_shape=[], output_shape=[50]
)
model_hub = keras.Sequential([
    hub_layer,
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])
model_hub.compile(loss="binary_crossentropy", optimizer="adam",
                  metrics=["accuracy"])
train_set = datasets["train"].batch(32).prefetch(1)
history = model_hub.fit(train_set, epochs=5)

5. Encoder–Decoder for Machine Translation and Beam Search

Bab ini membahas penerapan arsitektur encoder–decoder pada tugas Neural Machine Translation (NMT). Dalam arsitektur ini, encoder bertugas memproses kalimat sumber (misalnya bahasa Inggris) dan merangkum informasinya ke dalam representasi state. Selanjutnya, decoder menggunakan representasi tersebut untuk menghasilkan kalimat target (misalnya bahasa Prancis) secara autoregresif, yaitu satu kata pada setiap langkah waktu.

Proses Training dan Teacher Forcing

Selama proses pelatihan, digunakan teknik teacher forcing, di mana input ke decoder bukan hasil prediksi sebelumnya, melainkan kalimat target asli yang digeser satu langkah ke kanan. Proses decoding biasanya diawali dengan token khusus SOS (start-of-sequence) sebagai penanda awal kalimat.

Model dilatih menggunakan fungsi loss sparse_categorical_crossentropy, dengan tujuan memaksimalkan probabilitas distribusi kata target yang benar pada setiap time step.

Implementasi Encoder–Decoder

Untuk mempermudah implementasi, TensorFlow Addons menyediakan API seq2seq yang mendukung pembangunan arsitektur encoder–decoder secara modular. Beberapa komponen utama yang digunakan antara lain:

LSTMCell sebagai unit rekuren pada encoder dan decoder,

TrainingSampler untuk mengatur input decoder selama tahap training,

BasicDecoder sebagai pengendali alur proses decoding.

Pendekatan ini memungkinkan pemisahan yang jelas antara tahap encoding dan decoding, sehingga arsitektur model menjadi lebih fleksibel dan mudah dikembangkan.

Greedy Decoding vs Beam Search

Pada tahap inference, strategi paling sederhana adalah greedy decoding, yaitu memilih kata dengan probabilitas tertinggi pada setiap langkah. Namun, pendekatan ini sering menghasilkan terjemahan yang kurang optimal karena tidak mempertimbangkan konteks global kalimat.

Sebagai solusi, digunakan beam search, yaitu teknik decoding yang:

mempertahankan beberapa kandidat terjemahan terbaik secara bersamaan,

memperluas seluruh kandidat tersebut pada setiap langkah waktu,

kemudian hanya menyimpan beam_width urutan dengan probabilitas gabungan tertinggi.

Dengan mempertimbangkan beberapa kemungkinan urutan sekaligus, beam search mampu menghasilkan terjemahan yang lebih akurat, koheren, dan natural, terutama untuk kalimat yang panjang dan kompleks.

In [ ]:
import numpy as np
import tensorflow_addons as tfa
from tensorflow import keras
import tensorflow as tf

vocab_size = 20000
embed_size = 256
units = 512

encoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
decoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
sequence_lengths = keras.layers.Input(shape=[], dtype=np.int32)

embeddings = keras.layers.Embedding(vocab_size, embed_size)
encoder_emb = embeddings(encoder_inputs)
decoder_emb = embeddings(decoder_inputs)

encoder = keras.layers.LSTM(units, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_emb)
encoder_state = [state_h, state_c]

sampler = tfa.seq2seq.sampler.TrainingSampler()
decoder_cell = keras.layers.LSTMCell(units)
output_layer = keras.layers.Dense(vocab_size)

decoder = tfa.seq2seq.basic_decoder.BasicDecoder(
    decoder_cell, sampler, output_layer=output_layer
)
final_outputs, final_state, final_seq_lengths = decoder(
    decoder_emb, initial_state=encoder_state,
    sequence_length=sequence_lengths
)
Y_proba = tf.nn.softmax(final_outputs.rnn_output)

model = keras.Model(
    inputs=[encoder_inputs, decoder_inputs, sequence_lengths],
    outputs=[Y_proba]
)

# Beam search decoding (inference)
beam_width = 10
beam_decoder = tfa.seq2seq.beam_search_decoder.BeamSearchDecoder(
    cell=decoder_cell, beam_width=beam_width, output_layer=output_layer
)
encoder_state_beam = tfa.seq2seq.beam_search_decoder.tile_batch(
    encoder_state, multiplier=beam_width
)
start_tokens = tf.fill([batch_size], sos_id)
end_token = eos_id
outputs, _, _ = beam_decoder(
    decoder_emb, start_tokens=start_tokens, end_token=end_token,
    initial_state=encoder_state_beam
)


6. Attention Mechanisms and Transformer Architecture

Mekanisme attention memungkinkan decoder untuk secara dinamis memfokuskan perhatian pada bagian input yang paling relevan di setiap langkah waktu. Dengan cara ini, jalur informasi dari kata sumber ke kata target jadi jauh lebih pendek dibanding arsitektur encoder–decoder klasik. Dampaknya cukup besar: kualitas terjemahan meningkat signifikan, terutama saat menangani kalimat yang panjang dan kompleks.

Pada Bahdanau attention, yang juga dikenal sebagai additive atau concatenative attention, setiap output encoder dikombinasikan dengan hidden state decoder untuk menghitung skor kesesuaian
𝑒
(
𝑡
,
𝑖
)
e(t,i) menggunakan sebuah jaringan saraf kecil. Skor ini kemudian dinormalisasi dengan softmax untuk menghasilkan bobot perhatian
𝛼
(
𝑡
,
𝑖
)
α(t,i). Selanjutnya, context vector pada waktu
𝑡
t dihitung sebagai kombinasi linear dari seluruh output encoder yang ditimbang oleh bobot perhatian tersebut, lalu digabungkan dengan hidden state decoder untuk menghasilkan prediksi kata berikutnya.

Sebagai alternatif yang lebih efisien, Luong attention menggunakan pendekatan multiplicative dengan menghitung skor perhatian melalui dot product antara hidden state decoder dan output encoder, atau versi general yang menambahkan transformasi linear. Metode ini lebih ringan secara komputasi dan dalam banyak kasus justru memberikan performa yang lebih baik. TensorFlow Addons sendiri sudah menyediakan implementasi siap pakai seperti LuongAttention dan AttentionWrapper yang memudahkan integrasi attention ke dalam decoder.

Bab ini kemudian beralih ke Transformer, yaitu arsitektur yang sepenuhnya meninggalkan RNN dan CNN, dan hanya mengandalkan attention untuk memodelkan dependensi dalam data sekuens. Transformer mengombinasikan multi-head attention, feed-forward network yang diaplikasikan per posisi, layer normalization, residual connection, serta positional encoding berbasis fungsi sinus dan kosinus agar informasi urutan tetap terjaga.

Inti dari Transformer adalah scaled dot-product attention, yang menghitung perhatian dengan mengalikan query dan key, menskalakannya dengan akar dimensi key, lalu menerapkan softmax sebelum dikalikan dengan value. Pada multi-head attention, query, key, dan value diproyeksikan ke beberapa ruang representasi berbeda sehingga setiap head dapat mempelajari pola perhatian yang beragam, seperti hubungan posisi, struktur sintaks, maupun makna semantik. Hasil dari seluruh head kemudian digabungkan kembali. Berkat kemampuannya memodelkan dependensi jangka panjang secara paralel dan efisien, Transformer menjadi fondasi utama bagi hampir semua model NLP modern, termasuk BERT, GPT, dan sistem penerjemahan mutakhir.

In [ ]:
import tensorflow_addons as tfa
from tensorflow import keras
import tensorflow as tf
import numpy as np

# Luong Attention + AttentionWrapper
units = 512
encoder_outputs = ...              # [batch, time_src, units]
encoder_seq_len = ...              # [batch]
decoder_cell = keras.layers.LSTMCell(units)

attention_mech = tfa.seq2seq.attention_wrapper.LuongAttention(
    units, encoder_outputs,
    memory_sequence_length=encoder_seq_len
)
attn_cell = tfa.seq2seq.attention_wrapper.AttentionWrapper(
    decoder_cell, attention_mech, attention_layer_size=units
)

# Positional Encoding layer
class PositionalEncoding(keras.layers.Layer):
    def __init__(self, max_steps, max_dims, dtype=tf.float32, **kwargs):
        super().__init__(dtype=dtype, **kwargs)
        if max_dims % 2 == 1:
            max_dims += 1
        p, i = np.meshgrid(np.arange(max_steps), np.arange(max_dims // 2))
        pos_emb = np.empty((1, max_steps, max_dims))
        pos_emb[:, :, 0::2] = np.sin(p / 10000 ** (2 * i / max_dims)).T
        pos_emb[:, :, 1::2] = np.cos(p / 10000 ** (2 * i / max_dims)).T
        self.positional_embedding = tf.constant(pos_emb.astype(self.dtype))

    def call(self, inputs):
        shape = tf.shape(inputs)
        return inputs + self.positional_embedding[:, :shape[-2], :shape[-1]]

# Transformer-like skeleton using keras.layers.Attention
embed_size = 512; max_steps = 500; vocab_size = 10000
encoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
decoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)

embeddings = keras.layers.Embedding(vocab_size, embed_size)
enc_emb = embeddings(encoder_inputs)
dec_emb = embeddings(decoder_inputs)

pos_enc = PositionalEncoding(max_steps, embed_size)
encoder_in = pos_enc(enc_emb)
decoder_in = pos_enc(dec_emb)

Z = encoder_in
for _ in range(6):
    Z = keras.layers.Attention(use_scale=True)([Z, Z])
encoder_outputs = Z

Z = decoder_in
for _ in range(6):
    Z = keras.layers.Attention(use_scale=True, causal=True)([Z, Z])
    Z = keras.layers.Attention(use_scale=True)([Z, encoder_outputs])

outputs = keras.layers.TimeDistributed(
    keras.layers.Dense(vocab_size, activation="softmax")
)(Z)

transformer = keras.Model(
    inputs=[encoder_inputs, decoder_inputs],
    outputs=outputs
)


7. Recent Language Models: ELMo, ULMFiT, GPT, and BERT

Bagian penutup bab ini membahas lompatan besar di dunia Natural Language Processing pada rentang 2018–2019, periode ketika cara merepresentasikan dan memodelkan bahasa berubah drastis. Fokus utama pergeseran ini adalah pemanfaatan pretraining skala besar dan representasi bahasa yang jauh lebih kontekstual, sehingga model tidak lagi memahami kata secara terisolasi, melainkan sebagai bagian dari konteks kalimat yang utuh.

ELMo memperkenalkan konsep contextualized word embeddings, di mana representasi sebuah kata diambil dari state internal language model bidirectional yang dalam. Dengan pendekatan ini, satu kata yang sama bisa memiliki embedding berbeda tergantung konteks penggunaannya, sehingga makna ganda atau polisemik dapat ditangkap dengan jauh lebih baik dibanding embedding statis seperti Word2Vec atau GloVe.

ULMFiT menunjukkan bahwa language model berbasis LSTM yang dipretrain menggunakan self-supervised learning dapat di-fine-tune secara bertahap untuk berbagai tugas downstream. Strategi fine-tuning ini terbukti sangat efektif, bahkan ketika data berlabel terbatas, dan menjadi salah satu pijakan penting lahirnya konsep transfer learning modern di NLP.

GPT dan GPT-2 mengadopsi arsitektur Transformer decoder dengan masked multi-head attention dan dilatih sebagai language model autoregresif. Melalui pretraining skala besar, model ini mampu menunjukkan kemampuan zero-shot dan few-shot learning pada berbagai tugas NLP, hanya dengan mengandalkan pengetahuan bahasa yang telah dipelajari tanpa perlu pelatihan ulang yang berat.

BERT kemudian membawa pendekatan berbeda dengan menggunakan Transformer encoder bidirectional. Model ini dipretrain menggunakan dua objektif utama, yaitu Masked Language Model yang memprediksi token tersembunyi dari konteks dua arah, serta Next Sentence Prediction untuk mempelajari hubungan antar-kalimat. Hasilnya adalah representasi bahasa yang sangat kuat dan fleksibel, yang dapat di-fine-tune dengan ringan untuk beragam tugas seperti klasifikasi teks, question answering, dan natural language inference.

Secara keseluruhan, terobosan utama dari model-model ini terletak pada penggunaan tokenisasi berbasis subword, peralihan arsitektur dari LSTM ke Transformer, serta pemanfaatan self-supervised pretraining skala besar yang kemudian dapat diadaptasi secara efisien ke banyak tugas NLP downstream.